In [1]:
import scanpy as sc
import scipy.sparse as sp
import pandas as pd
import os
import numpy as np
import scvi
import seaborn as sns
from scipy.stats import median_abs_deviation

/Users/Shadi/Documents/projects/gen_analyzer/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"

ribo_genes = pd.read_table(ribo_url, skiprows=2, header = None)
ribo_genes

,0
0,FAU
1,MRPL13
2,RPL10
3,RPL10A
4,RPL10L
...,...
83,RPS9
84,RPSA
85,RSL24D1
86,RSL24D1P11


In [3]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [4]:
def pp(csv_path):
    adata = sc.read_csv(csv_path).T
    adata.X = sp.csr_matrix(adata.X)
    adata.obs['Sample'] = csv_path.split('_')[1].split('.')[0]
        
    sc.pp.filter_cells(adata, min_genes=200) #get rid of cells with fewer than 200 genes
    adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
    adata.var['ribo'] = adata.var_names.isin(ribo_genes[0].values)
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
    adata = adata[adata.obs.n_genes_by_counts < upper_lim]
    adata = adata[adata.obs.pct_counts_mt < 20]

    adata.obs["outlier"] = is_outlier(adata, "pct_counts_ribo", 5)
    adata = adata[~adata.obs["outlier"]]

    return adata
    

In [5]:
out = []

for file in os.listdir('../database/'):
    out.append(pp('../database/' + file))

/var/folders/g1/vddjmx092w30hswgd6ghnkjr0000gn/T/ipykernel_61638/3493494716.py:14: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["outlier"] = is_outlier(adata, "pct_counts_ribo", 5)
/var/folders/g1/vddjmx092w30hswgd6ghnkjr0000gn/T/ipykernel_61638/3493494716.py:14: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["outlier"] = is_outlier(adata, "pct_counts_ribo", 5)
/var/folders/g1/vddjmx092w30hswgd6ghnkjr0000gn/T/ipykernel_61638/3493494716.py:14: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["outlier"] = is_outlier(adata, "pct_counts_ribo", 5)
/var/folders/g1/vddjmx092w30hswgd6ghnkjr0000gn/T/ipykernel_61638/3493494716.py:14: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["outlier"] = is_outlier(adata, "pct_counts_ribo", 5)
/var

In [6]:
adata = sc.concat(out, join="outer")

In [7]:
adata

AnnData object with n_obs × n_vars = 24470 × 33868
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'outlier'

In [8]:
sc.pp.filter_genes(adata, min_cells=100)
adata

AnnData object with n_obs × n_vars = 24470 × 13804
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'outlier'
    var: 'n_cells'

In [9]:
adata.write_h5ad('combined.h5ad')

In [10]:
adata.obs.groupby('Sample').count()

/var/folders/g1/vddjmx092w30hswgd6ghnkjr0000gn/T/ipykernel_61638/2340059915.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adata.obs.groupby('Sample').count()


,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo,outlier
Sample,,,,,,,,
NB02,1328,1328,1328,1328,1328,1328,1328,1328
NB11,9930,9930,9930,9930,9930,9930,9930,9930
NB16,2707,2707,2707,2707,2707,2707,2707,2707
NB21,297,297,297,297,297,297,297,297
NB26,5182,5182,5182,5182,5182,5182,5182,5182
NB34,5026,5026,5026,5026,5026,5026,5026,5026


In [11]:
adata.layers['counts'] = adata.X.copy()

In [12]:
sc.pp.normalize_total(adata, target_sum = 1e4)
sc.pp.log1p(adata)
adata.raw = adata

In [13]:
adata

AnnData object with n_obs × n_vars = 24470 × 13804
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'outlier'
    var: 'n_cells'
    uns: 'log1p'
    layers: 'counts'

In [14]:
scvi.model.SCVI.setup_anndata(adata, layer = "counts", categorical_covariate_keys=["Sample"], 
                              continuous_covariate_keys=['pct_counts_mt', 'total_counts', 'pct_counts_ribo'])

In [15]:
print(scvi.__version__)

1.1.2


['/Users/Shadi/Documents/projects/gen_analyzer/venv/lib/python3.11/site-packages/scvi']

In [16]:
model = scvi.model.SCVI(adata)

In [17]:
model.train() 

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/Shadi/Documents/projects/gen_analyzer/venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 5/327:   1%|          | 4/327 [05:58<8:04:51, 90.07s/it, v_num=1, train_loss_step=3.72e+3, train_loss_epoch=3650.0] 

/Users/Shadi/Documents/projects/gen_analyzer/venv/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


: 